In [ ]:
Create a 12 nodes network in netsquid, with two repeters and one router, in a ring topology

In [ ]:
import netsquid as ns
from netsquid.nodes.node import Node
from netsquid.nodes.network import Network
from netsquid.nodes.connections import DirectConnection
from netsquid.protocols.routing import Router
from netsquid.nodes.qrepeater import QuantumRepeater

# Create the network
net = Network('Ring Network')

# Create the nodes
num_nodes = 12
nodes = [Node(f'Node{i}') for i in range(num_nodes)]

# Add the nodes to the network
for node in nodes:
    net.add_node(node)

# Create the repeaters
repeater1 = QuantumRepeater('Repeater1')
repeater2 = QuantumRepeater('Repeater2')

# Add the repeaters to the network
net.add_node(repeater1)
net.add_node(repeater2)

# Create the router
router = Router('Router')

# Add the router to the network
net.add_node(router)

# Connect the nodes in a ring topology
for i in range(num_nodes):
    node1 = nodes[i]
    node2 = nodes[(i + 1) % num_nodes]
    conn = DirectConnection(node1, node2)
    net.add_connection(conn)

# Connect the repeaters to the router
conn1 = DirectConnection(repeater1, router)
net.add_connection(conn1)
conn2 = DirectConnection(repeater2, router)
net.add_connection(conn2)

# Print the network information
ns.logger.info(net)


In this code, we first create a Network object and a list of Node objects. We then add the nodes to the network and create two QuantumRepeater objects and a Router object, which we also add to the network. We connect the nodes in a ring topology using DirectConnection objects, and connect the repeaters to the router with additional DirectConnection objects.

Finally, we print the network information using ns.logger.info(net). Note that you will need to have netsquid installed to run this code.

Create a random traffic and test it in the network

In [ ]:
import netsquid as ns
from netsquid.nodes.node import Node
from netsquid.nodes.network import Network
from netsquid.nodes.connections import DirectConnection
from netsquid.protocols.routing import Router
from netsquid.nodes.qrepeater import QuantumRepeater
from netsquid.protocols import NodeProtocol, Signals

class RandomTrafficProtocol(NodeProtocol):
    def __init__(self, node, dest, num_packets):
        super().__init__(node)
        self.dest = dest
        self.num_packets = num_packets
        self.packets_sent = 0
    
    def run(self):
        for i in range(self.num_packets):
            packet = f'Packet {i}'
            ns.logger.debug(f'{self.node.name} sending packet "{packet}" to {self.dest.name}')
            self.node.send_packet(packet, self.dest)
            self.packets_sent += 1
        
        self.node.ports[0].tx_output(Signals.SUCCESS, None)

# Create the network
net = Network('Ring Network')

# Create the nodes
num_nodes = 12
nodes = [Node(f'Node{i}') for i in range(num_nodes)]

# Add the nodes to the network
for node in nodes:
    net.add_node(node)

# Create the repeaters
repeater1 = QuantumRepeater('Repeater1')
repeater2 = QuantumRepeater('Repeater2')

# Add the repeaters to the network
net.add_node(repeater1)
net.add_node(repeater2)

# Create the router
router = Router('Router')

# Add the router to the network
net.add_node(router)

# Connect the nodes in a ring topology
for i in range(num_nodes):
    node1 = nodes[i]
    node2 = nodes[(i + 1) % num_nodes]
    conn = DirectConnection(node1, node2)
    net.add_connection(conn)

# Connect the repeaters to the router
conn1 = DirectConnection(repeater1, router)
net.add_connection(conn1)
conn2 = DirectConnection(repeater2, router)
net.add_connection(conn2)

# Create the protocol for random traffic
num_packets = 10
src = nodes[0]
dest = nodes[6]
protocol = RandomTrafficProtocol(src, dest, num_packets)

# Run the protocol
ns.run(protocol)

# Print the number of packets sent
ns.logger.info(f'{src.name} sent {protocol.packets_sent} packets to {dest.name}')


In this code, we first create a RandomTrafficProtocol class that generates random traffic between a source node and a destination node. We then create a Network object and a list of Node objects, add them to the network, and connect them in a ring topology. We also create two QuantumRepeater objects and a Router object, add them to the network, and connect them to the nodes.

We then create an instance of the RandomTrafficProtocol class and run it using ns.run(). After the protocol finishes running, we print the number of packets sent.

Note that in this example code, the RandomTrafficProtocol class sends packets using the Node.send_packet() method, which is a classical communication method. If you want to test quantum communication in the network, you will need to use quantum protocols instead.

Test this network using a  quantum routing protocol

In [ ]:
import netsquid as ns
from netsquid.nodes.node import Node
from netsquid.nodes.network import Network
from netsquid.nodes.connections import DirectConnection
from netsquid.protocols.routing import Router
from netsquid.nodes.qrepeater import QuantumRepeater
from netsquid.protocols import NodeProtocol, Signals
from netsquid.components.models.qerrormodels import FibreLossModel

class QuantumRoutingProtocol(NodeProtocol):
    def __init__(self, node, dest, num_qubits):
        super().__init__(node)
        self.dest = dest
        self.num_qubits = num_qubits
        self.qubits_sent = 0
    
    def run(self):
        for i in range(self.num_qubits):
            qubit = ns.qubits.create_qubit(self.node.qmemory)
            ns.logger.debug(f'{self.node.name} sending qubit {i} to {self.dest.name}')
            self.node.qmemory.put(qubit)
            self.node.send_qubit(qubit, self.dest)
            self.qubits_sent += 1
        
        self.node.ports[0].tx_output(Signals.SUCCESS, None)

# Create the network
net = Network('Ring Network')

# Create the nodes
num_nodes = 12
nodes = [Node(f'Node{i}') for i in range(num_nodes)]

# Add the nodes to the network
for node in nodes:
    net.add_node(node)

# Create the repeaters
repeater1 = QuantumRepeater('Repeater1')
repeater2 = QuantumRepeater('Repeater2')

# Add the repeaters to the network
net.add_node(repeater1)
net.add_node(repeater2)

# Create the router
router = Router('Router')

# Add the router to the network
net.add_node(router)

# Connect the nodes in a ring topology
for i in range(num_nodes):
    node1 = nodes[i]
    node2 = nodes[(i + 1) % num_nodes]
    loss_model = FibreLossModel(p_loss=0.1)  # set a loss model for the quantum channel
    conn = DirectConnection(node1, node2, models={'qubit_loss_model': loss_model})
    net.add_connection(conn)

# Connect the repeaters to the router
conn1 = DirectConnection(repeater1, router)
net.add_connection(conn1)
conn2 = DirectConnection(repeater2, router)
net.add_connection(conn2)

# Create the protocol for quantum routing
num_qubits = 10
src = nodes[0]
dest = nodes[6]
protocol = QuantumRoutingProtocol(src, dest, num_qubits)

# Run the protocol
ns.run(protocol)

# Print the number of qubits sent
ns.logger.info(f'{src.name} sent {protocol.qubits_sent} qubits to {dest.name}')


In this code, we create a QuantumRoutingProtocol class that generates quantum traffic between a source node and a destination node. We then create a Network object and a list of Node objects, add them to the network, and connect them in a ring topology with a loss model for the quantum channel. We also create two QuantumRepeater objects and a Router object, add them to the network, and connect them to the nodes.

We then create an instance of the QuantumRoutingProtocol class and run it using ns.run(). After the protocol finishes running, we print the number of qubits sent.

Note that in this example code, the QuantumRoutingProtocol class sends qubits using the `Node